In [1]:
# nuclio: ignore
import nuclio

In [2]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx


def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [3]:
# nuclio: end-code

### mlconfig

In [4]:
from mlrun import mlconf

In [5]:
mlconf.dbpath = mlconf.dbpath or './'
mlconf.dbpath

'http://mlrun-api:8080'

In [6]:
vcs_branch = 'development'
base_vcs = f'https://raw.githubusercontent.com/mlrun/functions/{vcs_branch}/'

mlconf.hub_url = mlconf.hub_url or base_vcs + f'{name}/function.yaml'
mlconf.hub_url

'/User/repos/functions/{name}/function.yaml'

In [7]:
import os
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'
mlconf.artifact_path

'/User/artifacts'

### save

In [8]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset", kind="job", with_doc=True,
                      handler=load_dataset, 
                      image="mlrun/ml-models:0.4.6")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["datagen", "filesutils"]
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-25 23:00:49,186 saving function: load-dataset, tag: latest
[mlrun] 2020-04-25 23:00:49,222 function spec saved to path: function.yaml


## tests

In [9]:
from mlrun import import_function

func = import_function("hub://load_dataset")

if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    func.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    func.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [10]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "wine"}}

### run remotely

In [11]:
run = func.run(NewTask(**task_params), artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-25 23:00:49,288 starting run tasks load toy dataset uid=3af3bd02284942a1baf0a11bd0b97be7  -> http://mlrun-api:8080
[mlrun] 2020-04-25 23:00:49,384 Job is running in the background, pod: tasks-load-toy-dataset-p8xcd
[mlrun] 2020-04-25 23:00:53,331 log artifact wine at /User/artifacts/wine.parquet, size: 17997, db: Y

[mlrun] 2020-04-25 23:00:53,362 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...d0b97be7,0,Apr 25 23:00:52,completed,tasks load toy dataset,host=tasks-load-toy-dataset-p8xcdkind=jobowner=adminv3io_user=admin,,dataset=wine,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3af3bd02284942a1baf0a11bd0b97be7  , !mlrun logs 3af3bd02284942a1baf0a11bd0b97be7 
[mlrun] 2020-04-25 23:00:55,611 run executed, status=completed


### or locally

In [12]:
from mlrun import run_local

In [13]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset}, artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-25 23:00:55,632 starting run mlrun-91976f-load_dataset uid=8fb4456d05ef43588e538da77231cdc7  -> http://mlrun-api:8080
[mlrun] 2020-04-25 23:00:57,708 log artifact wine at /User/artifacts/wine.parquet, size: 18454, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...7231cdc7,0,Apr 25 23:00:55,completed,mlrun-91976f-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-5859859b4f-hwhxd,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 8fb4456d05ef43588e538da77231cdc7 --project default , !mlrun logs 8fb4456d05ef43588e538da77231cdc7 --project default
[mlrun] 2020-04-25 23:00:57,764 run executed, status=completed
[mlrun] 2020-04-25 23:00:57,771 starting run mlrun-591b71-load_dataset uid=aaca6c70b7f04263afdb111818baf049  -> http://mlrun-api:8080
[mlrun] 2020-04-25 23:00:57,848 log artifact iris at /User/artifacts/iris.parquet, size: 5209, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...18baf049,0,Apr 25 23:00:57,completed,mlrun-591b71-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-5859859b4f-hwhxd,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run aaca6c70b7f04263afdb111818baf049 --project default , !mlrun logs aaca6c70b7f04263afdb111818baf049 --project default
[mlrun] 2020-04-25 23:00:57,888 run executed, status=completed
[mlrun] 2020-04-25 23:00:57,898 starting run mlrun-9d233c-load_dataset uid=54ef3611c8a6497fa239a85799f7ec3b  -> http://mlrun-api:8080
[mlrun] 2020-04-25 23:00:58,089 log artifact breast_cancer at /User/artifacts/breast_cancer.parquet, size: 147746, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...99f7ec3b,0,Apr 25 23:00:57,completed,mlrun-9d233c-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-5859859b4f-hwhxd,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run 54ef3611c8a6497fa239a85799f7ec3b --project default , !mlrun logs 54ef3611c8a6497fa239a85799f7ec3b --project default
[mlrun] 2020-04-25 23:00:58,176 run executed, status=completed
